In [ ]:
import cv2
import time
import serial
import mediapipe as mp


cap = cv2.VideoCapture(0)
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.7)


arduino = serial.Serial('/dev/tty.usbmodem12401', 9600)


Kp_x, Ki_x, Kd_x = 0.6, 0.0, 0.1
Kp_y, Ki_y, Kd_y = 0.6, 0.0, 0.1


deadband_threshold_x = 50
deadband_threshold_y = 20


max_control_change_x = 10.0
max_control_change_y = 10.0


control_signal_x_history = []
control_signal_y_history = []


integral_x, previous_error_x = 0, 0
integral_y, previous_error_y = 0, 0
previous_control_x, previous_control_y = 0, 0

previous_time = time.time()


def pid_control(error, integral, previous_error, Kp, Ki, Kd, dt):
    P = Kp * error
    integral += error * dt
    I = Ki * integral
    derivative = (error - previous_error) / dt
    D = Kd * derivative
    control_signal = P + I + D
    return control_signal, integral, error


while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to read from the webcam.")
        break

    
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_detection.process(frame_rgb)
    frame_height, frame_width = frame.shape[:2]

    if results.detections:
        for detection in results.detections:
            bboxC = detection.location_data.relative_bounding_box
            x = int(bboxC.xmin * frame_width)
            y = int(bboxC.ymin * frame_height)
            w = int(bboxC.width * frame_width)
            h = int(bboxC.height * frame_height)

            
            face_center_x = x + w // 2
            face_center_y = y + h // 2

            
            frame_center_x = frame_width // 2
            frame_center_y = frame_height // 2

            
            error_x = frame_center_x - face_center_x
            error_y = frame_center_y - face_center_y

            
            if abs(error_x) < deadband_threshold_x:
                error_x = 0
            if abs(error_y) < deadband_threshold_y:
                error_y = 0

            
            current_time = time.time()
            dt = current_time - previous_time
            previous_time = current_time

            
            if error_x != 0:
                control_signal_x, integral_x, previous_error_x = pid_control(
                    error_x, integral_x, previous_error_x, Kp_x, Ki_x, Kd_x, dt
                )
                control_signal_x = max(min(control_signal_x, previous_control_x + max_control_change_x),
                                       previous_control_x - max_control_change_x)
                control_signal_x_history.append(control_signal_x)
                if len(control_signal_x_history) > 5:
                    control_signal_x_history.pop(0)
                smoothed_control_signal_x = sum(control_signal_x_history) / len(control_signal_x_history)
                previous_control_x = smoothed_control_signal_x

            
            if error_y != 0:
                control_signal_y, integral_y, previous_error_y = pid_control(
                    error_y, integral_y, previous_error_y, Kp_y, Ki_y, Kd_y, dt
                )
                control_signal_y = max(min(control_signal_y, previous_control_y + max_control_change_y),
                                       previous_control_y - max_control_change_y)
                control_signal_y_history.append(control_signal_y)
                if len(control_signal_y_history) > 5:
                    control_signal_y_history.pop(0)
                smoothed_control_signal_y = sum(control_signal_y_history) / len(control_signal_y_history)
                previous_control_y = smoothed_control_signal_y

            
            arduino.write(f"X{smoothed_control_signal_x:.2f}Y{smoothed_control_signal_y:.2f}\n".encode())

            
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            cv2.line(frame, (frame_center_x, 0), (frame_center_x, frame_height), (0, 255, 0), 2)
            cv2.line(frame, (0, frame_center_y), (frame_width, frame_center_y), (0, 255, 0), 2)

    
    cv2.imshow('Face Tracker', frame)
    if cv2.waitKey(1) & 0xFF == ord('x'):
        break


cap.release()
cv2.destroyAllWindows()
arduino.close()